In [ ]:
import pandas as pd
import os
import shutil
import zipfile
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from scipy.integrate import simps
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from baysian import *
import math
import itertools
pd.set_option('display.max_columns', None)

In [ ]:
val = pd.read_csv('./csv/val_result_merge.csv')
test = pd.read_csv('./csv/bayesian_test.csv')

In [ ]:
val.dropna()

In [ ]:
test.dropna()

In [ ]:
val = val.dropna()
val['true'] = 0
val.loc[val['treated_t'] >= 1, 'true'] = 1
val['TF'] = 0
val['pred_class'] = 0
val.loc[val['pred1'] <= 0.5, 'pred_class'] = 1
val.loc[val['true'] == val['pred_class'], 'TF'] = 1
val.describe()
# val

In [ ]:
test = test.dropna()
test['true'] = 0
test.loc[test['treated_t'] >= 1, 'true'] = 1
test['TF'] = 0
test['pred_class'] = 0
test.loc[test['pred1'] <= 0.5, 'pred_class'] = 1
test.loc[test['true'] == test['pred_class'], 'TF'] = 1
test.describe()
# test

In [ ]:
val['pred'] = val['pred1']
test['pred'] = test['pred1']
test

In [ ]:
colname = "train_num_raw_t"

val_co = val.query('TF == 1')
val_in = val.query('TF == 0')
max_, min_ = get_max_min(val, colname)
co_, in_, val = normalize_row(val_co, val_in, val, colname, max_, min_)

plot_hist(co_, in_, colname, bins=50)

dd = plot_kde_hist(co_, in_, val, colname, 0.05, colname, upper_num=10000)

x, co_y_lst = plot_kde(co_, colname, 0.05, colname, bins=50, TF=1)
x, in_y_lst = plot_kde(in_, colname, 0.05, colname, bins=50, TF=0)

# # def integral_df(df,colname, max_, min_, x_lst, co_y_lst, in_y_lst):
test = integral_df(test, colname, max_, min_, x, co_y_lst, in_y_lst)


test = update_baysian(test['pred'], 
                      "normal_" + colname + "_m1", 
                      "normal_" + colname + "_m3", 
                      "pred_" + colname + "_after",
                      test)

In [ ]:

COL = ['raw_t', 
       'raw_pH', 
       'tem', 
       'MC_std']


for colname in COL:
       print('colname', colname)
       if colname[1] == 'MC_std':
              num = 0.01
       elif colname[1] == 'raw_t':
              num = 0.02
       else:
              num = 0.05
       val = pd.read_csv('./csv/val_result_merge.csv')
       test = pd.read_csv('./csv/bayesian_test.csv')

       val_co = val.query('TF == 1')
       val_in = val.query('TF == 0')
       max_, min_ = get_max_min(val, colname)
       co_, in_, val = normalize_row(val_co, val_in, val, colname, max_, min_)

       plot_hist(co_, in_, colname, bins=50, num=num)

       x, co_y_lst = plot_kde2(co_, colname, num, colname, bins=50, TF=1)
       x, in_y_lst = plot_kde2(in_, colname, num, colname, bins=50, TF=0)

       test = integral_df(test, colname, max_, min_, x, co_y_lst, in_y_lst)

       test = update_baysian(test['pred'],                      
                            "normal_" + colname + "_m1", 
                            "normal_" + colname + "_m3", 
                            "pred_" + colname + "_after",
                            test)

       test.to_csv('./csv/bayesian_test.csv')






In [ ]:
import math
import itertools

# COL = ['tem', 'raw_t', 'raw_pH', 'MC_std', 'gau_std']
COL = ['tem', 'raw_t', 'raw_pH', 'MC_std']

num_n = [2, 3, 4]

for num in num_n:
    print('num', num)
    val = pd.read_csv('./csv/val_result_merge.csv')
    test = pd.read_csv('./csv/bayesian_test.csv')
    for colname in tqdm(itertools.permutations(COL, num), total=len(COL)):

        if num == 1:
            test = update_baysian(test['before'],                      
                                "normal_" + colname[0] + "_m1", 
                                "normal_" + colname[0] + "_m3",
                                "pred_" + colname[0] + "_after",
                                test)

        elif num == 2:
            test = update_baysian(test['pred_' + colname[0] + '_after'],                      
                                "normal_" + colname[1] + "_m1", 
                                "normal_" + colname[1] + "_m3",
                                "pred_" + colname[0] + '_' + colname[1] + "_after",
                                test)

        elif num == 3:
            test = update_baysian(test['pred_' + colname[0] + '_' + colname[1] + '_after'],                      
                                "normal_" + colname[2] + "_m1", 
                                "normal_" + colname[2] + "_m3",
                                "pred_" + colname[0] + '_' + colname[1] + '_' + colname[2] + "_after",
                                test)

        elif num == 4:
            test = update_baysian(test['pred_' + colname[0] + '_' + colname[1] + '_' + colname[2] + '_after'],                      
                                "normal_" + colname[3] + "_m1", 
                                "normal_" + colname[3] + "_m3",
                                "pred_" + colname[0] + '_' + colname[1] + '_' + colname[2] + '_' + colname[3] + "_after",
                                test)

    test.to_csv('./csv/bayesian_test.csv')


